# 使用 FastText 做情感分析

在上一个 notebook 中，我们使用所有用于情绪分析的常用技术，成功地实现了约84%的测试准确率。在本笔记本中，我们将实现一个模型，该模型可以获得可比较的结果，同时训练速度显著加快，并使用大约一半的参数。更具体地说，我们将实现论文中的“FastText”模型[高效文本分类的技巧包](https://arxiv.org/abs/1607.01759).

## 准备数据

FastText论文中的一个关键概念是，它们计算输入句子的n元，并将其附加到句子的末尾。在这里，我们将使用双克。简单地说，双格是在一个句子中连续出现的一对单词/标记。

比如, 在句子中 "how are you ?",  bi-grams: "how are", "are you" and "you ?".

`generate_bigrams` 函数获取一个已标记的句子，计算bigram并将其附加到标记化列表的末尾。

In [ ]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(" ".join(n_gram))
    return x

例如：

In [ ]:
generate_bigrams(["This", "film", "is", "terrible"])

['This', 'film', 'is', 'terrible', 'film is', 'This film', 'is terrible']

TorchText `Field` 有一个 `preprocessing` 参数。此处传递的函数将在句子被标记化（从字符串转换为标记列表）之后，但在其被数字化（从标记列表转换为索引列表）之前应用于句子。这就是我们传递 `generate_bigrams` 函数的地方。

由于我们不使用RNN，我们不能使用压缩填充序列，因此我们不需要设置 `include_length=True`。

In [ ]:
import torch
from torchtext.legacy import data, datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(
    tokenize="spacy",
    tokenizer_language="en_core_web_sm",
    preprocessing=generate_bigrams,
)

LABEL = data.LabelField(dtype=torch.float)

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


As before, we load the IMDb dataset and create the splits.

In [ ]:
import random

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Build the vocab and load the pre-trained word embeddings.

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(
    train_data,
    max_size=MAX_VOCAB_SIZE,
    vectors="glove.6B.100d",
    unk_init=torch.Tensor.normal_,
)

LABEL.build_vocab(train_data)

并创建迭代器。

In [ ]:
BATCH_SIZE = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device
)

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## 构建模型


该模型的参数远小于前一模型，因为它只有两个具有任何参数的层，嵌入层和线性层。看不到RNN组件！

相反，它首先使用 `Embedding`  层（蓝色）计算每个单词的单词嵌入，然后计算所有单词嵌入的平均值（粉色），并通过 `Linear` 层（银色）将其输入，就这样！

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment8.png?raw=1)

我们使用 `avg_pool2d`（平均池二维）函数实现平均。最初，您可能会认为使用二维池似乎很奇怪，我们的句子肯定是一维的，而不是二维的？但是，您可以将单词嵌入视为二维网格，其中单词沿着一个轴，单词嵌入的维度沿着另一个轴。下图是一个转换为5维单词嵌入后的示例句子，单词沿垂直轴，嵌入沿水平轴。[4x5]张量中的每个元素都由一个绿色块表示。

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment9.png?raw=1)

`avg_pool2d` 使用大小为“嵌入式”的过滤器。形状[1]（即句子的长度）乘以1。这在下图中以粉红色显示。

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment10.png?raw=1)

我们计算过滤器覆盖的所有元素的平均值，然后过滤器向右滑动，计算句子中每个单词的下一列嵌入值的平均值。

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment11.png?raw=1)

每个过滤器位置为我们提供一个值，即所有覆盖元素的平均值。在滤波器覆盖所有嵌入维数后，我们得到[1x5]张量。这个张量然后通过线性层来产生我们的预测。

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):

        # text = [sent len, batch size]

        embedded = self.embedding(text)

        # embedded = [sent len, batch size, emb dim]

        embedded = embedded.permute(1, 0, 2)

        # embedded = [batch size, sent len, emb dim]

        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)

        # pooled = [batch size, embedding_dim]

        return self.fc(pooled)

如前所述，我们将创建 `FastText` 类的实例。

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

查看我们模型中的参数数量，我们可以看到，我们的参数与第一个笔记本中的标准RNN大致相同，只有前一个模型的一半。

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 2,500,301 trainable parameters


并将预训练的向量复制到我们的嵌入层。

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1606, -0.7357,  0.5809,  ...,  0.8704, -1.5637, -1.5724],
        [-1.3126, -1.6717,  0.4203,  ...,  0.2348, -0.9110,  1.0914],
        [-1.5268,  1.5639, -1.0541,  ...,  1.0045, -0.6813, -0.8846]])

并将预训练的向量复制到我们的嵌入层。

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## 训练模型

训练模型与上次完全相同。

我们初始化优化器。。。

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

我们定义标准，并将模型和标准放置在GPU上（如果可用）。。。

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

我们实现了计算精度的函数。。。

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()  # convert into float for division
    acc = correct.sum() / len(correct)
    return acc

我们定义了一个用于训练模型的函数...


**Note**: 我们不再使用 dropout，因此不需要使用 `model.train（）`，但正如第一本笔记本中提到的，使用它是一个很好的实践。

In [ ]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

我们定义了一个用于测试模型的函数...

**Note**: 再次，我们离开 `model.eval（）` 即使我们不使用dropout。

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

如前所述，我们将实现一个有用的函数来告诉我们一个epoch需要多长时间。

In [ ]:
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

最后，我们训练我们的模型。

In [ ]:
N_EPOCHS = 5

best_valid_loss = float("inf")

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut3-model.pt")

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%")
    print(f"\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%")

/home/ben/miniconda3/envs/pytorch17/lib/python3.8/site-packages/torchtext-0.9.0a0+c38fd42-py3.8-linux-x86_64.egg/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 0.688 | Train Acc: 61.31%
	 Val. Loss: 0.637 |  Val. Acc: 72.46%
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 0.651 | Train Acc: 75.04%
	 Val. Loss: 0.507 |  Val. Acc: 76.92%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 0.578 | Train Acc: 79.91%
	 Val. Loss: 0.424 |  Val. Acc: 80.97%
Epoch: 04 | Epoch Time: 0m 6s
	Train Loss: 0.501 | Train Acc: 83.97%
	 Val. Loss: 0.377 |  Val. Acc: 84.34%
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.435 | Train Acc: 86.96%
	 Val. Loss: 0.363 |  Val. Acc: 86.18%


…并获得测试精度！

结果与上一个笔记本中的结果相当，但培训所需时间要少得多！

In [ ]:
model.load_state_dict(torch.load("tut3-model.pt"))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f"Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%")

Test Loss: 0.381 | Test Acc: 85.42%


## 用户输入

和以前一样，我们可以测试用户提供的任何输入，确保从标记化的句子中生成双字图。

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")


def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

一个负面评论的例子...

In [ ]:
predict_sentiment(model, "This film is terrible")

2.1313092350011553e-12

一个正面评论的例子...

In [ ]:
predict_sentiment(model, "This film is great")

1.0